In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
from datetime import date
import dash_bootstrap_components as dbc

In [2]:
SIDEBAR_STYLE = {
    "position":"fixed",
    "top": "0rem",
    "left": "0rem",
    "bottom": "0rem",
    "width": "18rem",
    "padding": "2rem 1rem",
    "background-color": "#1c3242",
}

In [3]:
df = pd.read_csv("FAOSTAT_data_en_11-20-2022 (1).csv")

In [4]:
df.drop(["Item", "Item Code", "Area Code (M49)", "Domain Code", "Domain", "Flag", "Note", "Unit","Year Code", "Element Code", "Flag Description"], axis = 1, inplace = True)
df["Value"] = df["Value"]*1000
df["Total"] = df["Value"][df["Element"] == "Total Population - Both sexes"]
df["Males"] = df["Value"][df["Element"] == "Total Population - Male"]
df["Females"] = df["Value"][df["Element"] == "Total Population - Female"]
df["Rural"] = df["Value"][df["Element"] == "Rural population"]
df["Urban"] = df["Value"][df["Element"] == "Urban population"]
df["Males"] = df["Males"].shift(-1)
df["Females"] = df["Females"].shift(-2)
df["Rural"] = df["Rural"].shift(-3)
df["Urban"] = df["Urban"].shift(-4)
df_mod = df.dropna(axis = 0)
df_mod.drop(axis = 1, columns = "Element", inplace = True)

C:\Users\mosta\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [5]:
df_2 = pd.read_csv("API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4701247.csv")

In [6]:
app = Dash(external_stylesheets = [dbc.themes.SLATE])

In [7]:
navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Page 1", href="#")),
        dbc.NavItem(dbc.NavLink("Page 2", href="#")),
    ],
    brand="My Dashboard",
    brand_href="#",
    color="primary",
    dark=True,
    style = {"height": "3.5rem"}
)

In [8]:
sidebar = html.Div([
                    html.H2("Selection Sidebar", style = {"textAlign": "left", "color": "white"}),
                    html.Hr(),
                    html.H3("Select countries", style = {'marginBottom': '1em', "fontSize": 14}),
                    html.Div([dcc.Dropdown(df["Area"].unique(), ["Argentina"], multi = True, id = "dropdown-1")], style = {'marginBottom': '2em', "fontSize": 14}),
                    html.H3("Select category", style = {'marginBottom': '1em', "fontSize": 14}),
                    html.Div([dcc.Dropdown(["Total", "Males", "Females", "Urban", "Rural"], "Total", multi = False, id = "dropdown-2")], style = {'marginBottom': '2em', "fontSize": 14}),
                    html.H3("Select time range", style = {'marginBottom': '1em', "fontSize": 14}),
                    html.Div([dcc.RangeSlider(1950, 2022, 10, marks = {i: f"{i}" for i in range(1950, 2022, 10)}, value = [2000, 2010], id='my-range-slider')], style = {'marginBottom': '2em'}),
                    #dbc.Row([dbc.Button("View Results!", id = "my-button")], justify="center")
], style = SIDEBAR_STYLE)

In [9]:
df_3 = df_2.set_index("Country Name").T

In [10]:
app.layout = html.Div([dbc.Container([
                       #dbc.Row([navbar]),
                       html.Br(),
                       dbc.Row([dbc.Col(sidebar, width = 3),dbc.Col(html.H1("World Population Dashboard", style = {"textAlign": "center", "color": "white"}), width = 9)]),
                       html.Br(),
                       html.Br(),   
                       dbc.Row([dbc.Col(width = 3),dbc.Col([dcc.Graph(id = "graph")], width = 9)]),
                       html.Br(),   
                       dbc.Row([dbc.Col(width = 3),dbc.Col([dcc.Graph(id = "graph_4")], width = 9)]),
                       html.Br(),
                       dbc.Row([dbc.Col(width = 3), 
                                dbc.Col([html.Div([dcc.Graph(id = "graph_2")], style = {"display": "inline-block","width": "42%", "float": "left", "marginLeft": 2}),
                                         html.Div([dcc.Graph(id = "graph_3")], style = {"display": "inline-block","width": "42%", "float": "right", "marginRight": 2})], width = 9)
                               ]),], fluid = True)],style = {"backgroundColor": "#14232e", "marginTop": 0, "marginBottom": 0, "marginRight": 0, "marginLeft": 0} )

In [11]:
@app.callback(Output("graph", "figure"),
              [Input("dropdown-1", "value"),
              Input("dropdown-2", "value"),
              Input('my-range-slider', "value")])

def update_graph (dropdown1, dropdown2, rangeslider):   
    df_updated = df_mod[["Area", "Year", dropdown2]][(df_mod["Area"].isin(dropdown1)) &(df_mod["Year"] > (rangeslider[0]-1)) & (df_mod["Year"] < (rangeslider[1]+1))]
    #data =go.scatter(x = df_updated["Year"], y = df_updated[dropdown2])
    fig = px.line(df_updated, x = "Year", y = dropdown2, color = "Area", markers = False, title = "Population vs Time", color_discrete_sequence= px.colors.qualitative.Pastel)
    #fig = go.Figure(data = data)
    fig.update_layout(height = 390, paper_bgcolor = "#10212e", plot_bgcolor = "#10212e", title_y = 0.98, title_x = 0.5, font = {"color": "white"}, margin=dict(l=20, r=20, t=30, b=20))
    fig.update_xaxes(showgrid = False)
    fig.update_yaxes(showgrid = True, gridcolor = "#1F2A40")
    fig.update_traces(line=dict(width=2))
    return fig

@app.callback(Output("graph_4", "figure"),
              [Input("dropdown-1", "value"),
              Input('my-range-slider', "value")])

def update_graph_4 (dropdown_1, rangeslider):
    df_updated = df_2[df_2["Country Name"].isin(dropdown_1)]
    df_updated = df_updated.T
    df_updated.columns = df_updated.iloc[0]
    df_updated = df_updated.drop(df_updated.index[0], axis = 0)
    df_updated.index = df_updated.index.astype(int)
    df_updated = df_updated[(df_updated.index > (rangeslider[0]-1)) & (df_updated.index < (rangeslider[1]+1))]
    #data =go.scatter(x = df_updated["Year"], y = df_updated[dropdown2])
    fig = px.line(df_updated, markers = False, title = "GDP vs Time", color_discrete_sequence= px.colors.qualitative.Pastel,labels={'index': 'Year', 'value':'GDP'})
    #fig = go.Figure(data = data)
    fig.update_layout(height = 390, paper_bgcolor = "#10212e", plot_bgcolor = "#10212e", title_y = 0.98, title_x = 0.5, font = {"color": "white"}, margin=dict(l=20, r=20, t=30, b=20))
    fig.update_xaxes(showgrid = False)
    fig.update_yaxes(showgrid = True, gridcolor = "#1F2A40")
    fig.update_traces(line=dict(width=2))
    return fig


@app.callback(Output("graph_2", "figure"),
              Input("dropdown-1", "value"))

def update_graph_2(dropdown_1):
    df_updated = df_mod[(df_mod["Area"].isin(dropdown_1)) & (df_mod["Year"] == 2021)]
    fig = px.bar(df_updated, y = "Area", x = ["Males", "Females"], title = "Gender Composition", color_discrete_sequence= px.colors.qualitative.Pastel1)
    fig.update_layout(height = 380,paper_bgcolor = "#10212e", plot_bgcolor = "#10212e", title_y = 0.98, title_x = 0.5, font = {"color": "white"}, margin=dict(l=20, r=20, t=30, b=20))
    fig.update_xaxes(showgrid = False)
    fig.update_yaxes(showgrid = False)
    return fig


@app.callback(Output("graph_3", "figure"),
              Input("dropdown-1", "value"))

def update_graph_3(dropdown_1):
    df_updated = df_mod[(df_mod["Area"].isin(dropdown_1)) & (df_mod["Year"] == 2021)]
    fig = px.bar(df_updated, y = "Area", x = ["Rural", "Urban"], title = "Location Distribution", color_discrete_sequence= px.colors.qualitative.Set3)
    fig.update_layout(height = 380, paper_bgcolor = "#10212e", plot_bgcolor = "#10212e", title_y = 0.98, title_x = 0.5, font = {"color": "white"}, margin=dict(l=20, r=20, t=30, b=20))
    fig.update_xaxes(showgrid = False)
    fig.update_yaxes(showgrid = False)
    return fig

In [ ]:
if __name__ == "__main__":
    app.run_server(port = 8050)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Dec/2022 16:02:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:51] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:51] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:51] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:51] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 16:02:52] "POST /_dash-